# Método AHP

In [7]:
import numpy as np
from fractions import Fraction
import os

nodeDict = {}
leavesList = []
dirpath = os.getcwd()

def get_category(level):
    if level == 0:
        return "Objective"
    elif level == 1:
        return "Sub-Objective"
    elif level == 2:
        return "Criteria"
    elif level > 2:
        return "Sub-Criteria " + str(level - 2)
    
def calcMatrixWeights(matrix):
    transp = matrix.transpose()
    sums = list(map(lambda x: np.sum(x), transp))
    norm_transp = np.array(list(map(lambda x: x[0]/x[1], zip(transp, sums))))
    weights = np.array(list(map(lambda x: np.sum(x)/len(x), norm_transp.transpose())))
    
    return weights
    

class Candidate:
    def __init__(self, name):
        self.name = name
        self.values = {}
        self.score = None
    def add(self, name, value):
        self.values[name] = value
        
    
class HNode:
    def __init__(self, name, inv_prop = False):
        self.name = name
        self.category = "Objective"
        self.inv_prop = inv_prop
        self.level = 0
        self.children = []
        self.comparison_matrix = None
        self.weight = 1
        self.wmatrix = 1
        self.matrix_dim = 0
        self.criteria_scores = 0
        self.has_parent = False
        self.is_leaf = False
        
    def insert_children(self, children_list):
        for child in children_list:
            child.update_category(self)
            self.children.append(child)
            child.has_parent = True
            
    def update_category(self, parent):
        self.level = parent.level + 1
        self.category = get_category(self.level)
        for child in self.children:
            child.update_category(self)
            
    def set_children_weights(self):
        
        weights = calcMatrixWeights(self.comparison_matrix)
        
        for index, child in enumerate(self.children):
            child.weight = weights[index]
            
        self.wmatrix = np.matrix(weights).transpose()
        
    def score(self):
        if len(self.children) > 0:
            cMatrix = np.array([c.score() for c in self.children]).transpose()
            self.criteria_scores = np.array(np.matmul(cMatrix, self.wmatrix).transpose())[0]
            return self.criteria_scores
        elif self.name not in leavesList:
            print("Error! No criteria data for", self.name)
            exit()
        else:
            return self.criteria_scores
        
        
    def toStr(self, debugMode = False):
        string = ""
        for i in range(0, self.level):
            string = "    " + string
        string = string + self.name 
        if self.inv_prop: string = string + " ($)"
        if debugMode:
            string = string + " (" + str(hex(id(self))) + ") (is_leaf: " + str(self.is_leaf) + ")\n" # para debug
        else:
            string = string + " - " + self.category + " (Weight: " + "{:.2f}".format(self.weight) + ")\n"
        for child in self.children:
            string = string + child.toStr(debugMode)
        return string

def flagInconsistencies():
    
    ir = [0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51] # tabela de valores IR
    
    for node in nodeDict.values():
        if node.matrix_dim > 2:
            eigen, _ = np.linalg.eig(node.comparison_matrix)
            eigen = np.real(max(eigen))
            n = node.matrix_dim
            ic = (eigen - n)/(n - 1)
            rc = ic / ir[n-1]
            if rc >= 0.1:
                print("Warning: inconsistent matrix for", node.name, "detected with an RC of", rc, ".")
        elif node.matrix_dim == 0:
            if len(node.children) > 0:
                print("Error! ", node.name, "should have a comparison matrix!")
                exit()
            else:
                node.is_leaf = True
                leavesList.append(node.name)
    
def buildObj(objStr):
    
    inv_prop = False
    lineList = objStr.splitlines()
    objName = lineList.pop(0).strip()
    if objName[-1:] == "$":
        objName = objName[:-1]
        inv_prop = True
        
    if objName not in nodeDict:
        obj = HNode(objName, inv_prop)
        nodeDict[objName] = obj
    else:
        obj = nodeDict[objName]
        return obj
    for i, line in enumerate(lineList):
        if line[1] != " ":
            subLineList = [line[1:]]
            for subLine in lineList[i+1:]:
                if subLine[1] != " ":
                    break
                else:
                    subLineList.append(subLine[1:])
            obj.insert_children([buildObj("\n".join(subLineList))])
    return obj

def addMatrices():
    
    matrix_file = open(os.path.join(dirpath, "matrices.txt"), encoding = "utf-8")
    matrixStr = matrix_file.read()
    matrix_file.close()
    
    lineList = matrixStr.strip().splitlines()
    
    counter = 0
    currNodeName = lineList[counter]
    
    while counter <= len(lineList) - 1:
        
        currNodeName = lineList[counter]
        firstln = list(map(lambda x: float(Fraction(x)), lineList[counter + 1].split()))
        
        lineList[counter + 1] = lineList[counter + 1] + " (" + nodeDict[currNodeName].children[0].name + ")"
        
        if currNodeName in nodeDict:
            node = nodeDict[currNodeName]
            node.comparison_matrix = np.array([firstln])
            node.matrix_dim = len(firstln)
            for i in range(counter + 2, counter + 1 + node.matrix_dim):
                try:
                    ln = list(map(lambda x: float(Fraction(x)), lineList[i].split()))
                except:
                    print("Error! Non-square matrix detected at", currNodeName, "node!")
                    exit()
                else:
                    node.comparison_matrix = np.append(node.comparison_matrix, [ln], axis = 0)  
                    lineList[i] = lineList[i] + " (" + nodeDict[currNodeName].children[i - counter - 1].name + ")"
            node.set_children_weights()
        else:
            print("Warning: ", currNodeName, "not in hierarchy!")
            
        counter = counter + len(firstln) + 1
        
    return "\n".join(lineList)
                
def calcScore(candidate, objective):
    
    score = None
    
    if objective.is_leaf and objective.name in candidate.values:
        score = candidate.values[objective.name] * objective.weight
    elif objective.is_leaf:
        print("Warning! Score not found for", object.name, "in", candidate.name, ". Assuming one.")
        score = 1
    else:
        score = sum(list(map(lambda x: calcScore(candidate, x), objective.children)))
        
    return score

def normCriteria(candidateList):
    
    criteriaDict = {}
    
    for l in leavesList:
        criteriaMatrix = []
        normedScores = None
        for c in candidateList:
            if nodeDict[l].inv_prop:
                cRow = [candidateList[j].values[l]/c.values[l] for j in range(0, len(candidateList))]
            else:
                cRow = [c.values[l]/candidateList[j].values[l] for j in range(0, len(candidateList))]
            criteriaMatrix.append(cRow)
        normedScores = calcMatrixWeights(np.array(criteriaMatrix))
        
        for i, c in enumerate(candidateList):
            c.values[l] = normedScores[i]
        
        nodeDict[l].criteria_scores = normedScores

    
                
def calcCandidates(obj):
    
    candidateList = []
    
    cand_file = open(os.path.join(dirpath, "candidates.txt"), encoding = "utf-8")
    candStr = cand_file.read()
    cand_file.close()
    
    lineList = candStr.strip().splitlines()
    
    new_cand = True
    curr_cand = -1
    
    for i in range(0, len(lineList)):
        if new_cand:
            candidateList.append(Candidate(lineList[i].strip()))
            curr_cand + 1
            new_cand = False
        else:
            if lineList[i][0] == " ":
                line = list(map(lambda x: x.strip(), lineList[i].split(":")))
                if len(line) != 2:
                    print("Error retrieving values for", candidateList[curr_cand].name, "at line", i - 1)
                else:
                    value = float(Fraction(line[1]))
                    valueName = line[0]
                    candidateList[curr_cand].add(valueName, value)
            else:
                candidateList.append(Candidate(lineList[i].strip()))
                curr_cand + 1
                new_cand = False
    
    normCriteria(candidateList)
    
    return zip(candidateList, obj.score()), candStr

def printResults(result, obj, matrixStr, candStr):
    maxscore = 0
    maxcandidate = None
    print("====================HIERARCHY=====================")
    print(obj.toStr().strip())
    print("==============COMPARISON MATRICES=================")
    print(matrixStr.strip())
    print("=====================OPTIONS======================")
    print(candStr.strip())
    print("=====================SCORES=======================")
    for candidate, score in result:
        if maxscore < score:
            maxscore = score
            maxcandidate = candidate
        print(candidate.name, ": ", score)
    print("=====================RESULT=======================")
    print("Regarding '" + obj.name + "', the best possible \n"
        + "option may be '" + maxcandidate.name + "', with \n"
        + "a score of " + "{:.3f}".format(maxscore) + " obtained through the \n" 
        + "Analytic Hierarchy Process (AHP).")
    

In [8]:
def main():
    
    obj_file = open(os.path.join(dirpath, "hierarchy.txt"), "r", encoding = "utf-8")
    objStr = obj_file.read()
    obj_file.close()

    objective = buildObj(objStr)

    matrixStr = addMatrices()

    flagInconsistencies()

    result, candStr = calcCandidates(objective)

    printResults(result, objective, matrixStr, candStr)


In [9]:
main()

====================HIERARCHY=====================
Comprar Carro Novo - Objective (Weight: 1.00)
    Preço - Sub-Objective (Weight: 0.74)
    Desempenho - Sub-Objective (Weight: 0.19)
        Potência - Criteria (Weight: 0.17)
        Consumo - Criteria (Weight: 0.83)
    Estilo - Sub-Objective (Weight: 0.08)
==============COMPARISON MATRICES=================
Desempenho
 1 1/5 (Potência)
 5 1 (Consumo)
Comprar Carro Novo
 1   5   8 (Preço)
 1/5 1   3 (Desempenho)
 1/8 1/3 1 (Estilo)
=====================OPTIONS======================
Carro 1
 Preço: 1
 Potência: 1
 Consumo: 1
 Estilo: 1
Carro 2
 Preço: 1/2
 Potência: 7
 Consumo: 11/18
 Estilo: 9
Carro 3
 Preço: 2/3
 Potência: 5
 Consumo: 17/18
 Estilo: 5
=====================SCORES=======================
Carro 1 :  0.4083812482694223
Carro 2 :  0.2699651261312027
Carro 3 :  0.32165362559937494
=====================RESULT=======================
Regarding 'Comprar Carro Novo', the best possible 
option may be 'Carro 1', with 
a score of 0